In [2]:
import os.path
from os import path
import pandas as pd


# 캐글 ↓제출경로
data_path = '/kaggle/input/bike-sharing-demand/'
if not path.exists(data_path):
    data_path = '../input/'
print(f'input directory path : {data_path}')

train = pd.read_csv(data_path + 'train.csv')
test = pd.read_csv(data_path + 'test.csv')
submission = pd.read_csv(data_path + 'sampleSubmission.csv')

print(f'train shape : {train.shape}')
print(f'test_shape : {test.shape}')

# 날씨 weather ==> 4 인것 제거,
train = train[train['weather'] != 4]

# train 과 test 데이터 합치기 (index 역시 합쳐야 한다.)

all_data = pd.concat([train, test], ignore_index=True)
all_data

from datetime import datetime

# 날짜 피처 생성
all_data['date'] = all_data['datetime'].apply(lambda x : x.split()[0])

# 연도 피처
all_data['year'] = all_data['datetime'].apply(lambda x: x.split()[0].split('-')[0])

# 월피처
all_data['month'] = all_data['datetime'].apply(lambda x: x.split()[0].split('-')[1])

# 시 피처 생성
all_data['hour'] = all_data['datetime'].apply(lambda x: x.split()[1].split(':')[0])

# 요일 피처 생성
all_data['weekday'] = all_data['date'].apply(lambda dateString : datetime.strptime(dateString, "%Y-%m-%d").weekday())

# 훈련데이터 매달 1일~19일, 테스트 데이터 매달 20일~ 이므로 day 사용필요 x 
# minute, second 역시 사용필요 x (상식적)
all_data.head()


# all_data['datetime'] = pd.to_datetime(all_data['datetime'])
# all_data['year'] = all_data['datetime'].dt.year , dt.month, dt.hour, dt.weekday 로 변경가능


# 필요 없는 피처 제거

# 1. casual 과 registered 피처는 테스트 데이터 에 없는 피처이므로 제거
# 2. datetime 피처는  date(year,month,day) 피처가 가지고 있기에 필요 x
# 3. season 피처가 month 대분류이기에 month 제거
# 4. windspeed 상관관계 약하기에 제거 한다 

drop_features = ['casual', 'registered', 'datetime', 'date', 'month', 'windspeed']
all_data = all_data.drop(drop_features, axis=1)

all_data

# 훈련 데이터와 테스트 데이터 나누기
X_train = all_data[~pd.isnull(all_data['count'])]
X_test = all_data[pd.isnull(all_data['count'])]

# 타깃값 count 제거
X_train = X_train.drop(['count'], axis=1)
X_test = X_test.drop(['count'], axis=1)

y = train['count'] # 타깃값 

X_train.head()

# 테스트 데이터 왜있는지....
# 평가 지표 계싼 함수 작성

import numpy as np

def rmsle(y_true, y_pred, convertExp=True):
    # 지수변환
    if convertExp:
        y_true = np.exp(y_true)
        y_pred = np.exp(y_pred)
        
    log_true = np.nan_to_num(np.log(y_true+1))
    log_pred = np.nan_to_num(np.log(y_pred+1))
    
    # RMSLE 계산
    output = np.sqrt(np.mean((log_true -log_pred)**2))
    return output

# sklearn.metrics.mean_squared_log_error 이용시 대체가능

input directory path : ../input/
train shape : (10886, 12)
test_shape : (6493, 9)


In [3]:
from sklearn.linear_model import LinearRegression

linear_reg_model = LinearRegression()

log_y = np.log(y) 
linear_reg_model.fit(X_train, log_y)
# closed form solution 

preds = linear_reg_model.predict(X_train)

print(f'sklearn 선형 회귀의 RMSLE 값 : {rmsle(log_y, preds, True):.4f}')
print(preds[:10])

sklearn 선형 회귀의 RMSLE 값 : 1.0205
[2.19541075 2.27905813 2.38251905 2.59669148 2.7001524  2.75393502
 2.79636272 2.77413081 3.11399607 3.3418234 ]


In [8]:
# dataframe -> numpy 로 변경해서 계산하기 쉽게 ㅇㅇ 
np_log_y = log_y.to_numpy().reshape(len(log_y),1)
np_x_train = X_train.to_numpy().astype(float)
# bias 값 추가 
insert_column = np.ones_like(np_log_y)

# bias 값 연결 
np_x_train = np.c_[np_x_train, insert_column]
# numpy 로 직접 계산 
w = np.linalg.inv(np_x_train.T @ np_x_train) @ np_x_train.T @ np_log_y
print(w.shape)
second_pred = np_x_train @ w
print(second_pred[:10].reshape(10))

(5, 11)
(11, 1)
[2.19541074 2.27905813 2.38251905 2.59669148 2.70015239 2.75393505
 2.79636272 2.7741308  3.11399606 3.3418234 ]
